In [1]:
import pandas as pd
import matplotlib.pyplot as plt
# Abrir el archivo xlsx
df = pd.read_excel('../data/data.xlsx')

In [2]:
# Contar los nan en cada columna
df.isna().sum()
cols = df.columns
# Cambiar atributos "hogar" de Categoria 2 a "hogar2"
df.loc[df['Categoria 2'] == 'hogar', 'Categoria 2'] = 'hogar2'
# Contar los 'hogar' en categoria 2
df.loc[df['Categoria 2'] == 'hogar2', 'Categoria 2'].count()
# Obtener links entre categorias 1 y 2
df12 = df[['Categoria 1', 'Categoria 2']]
df12.loc[df12['Categoria 2'].isna(), 'Categoria 2'] = df12['Categoria 1']+ '2'
df12 = df12.groupby(['Categoria 1', 'Categoria 2']).size().reset_index(name='counts')
# Obtener links entre categorias 2 y 3
df23 = df[['Categoria 2', 'Categoria 3']]
# df23 = df23.dropna()
# Si Categoria 3 es nan, reemplazar por Categoria 2
df23.loc[df23['Categoria 3'].isna(), 'Categoria 3'] = df23['Categoria 2']+ '3'
df23 = df23.groupby(['Categoria 2', 'Categoria 3']).size().reset_index(name='counts')
# Obtener todas las categorias
categorias11 = df12['Categoria 1'].unique().tolist()
categorias12 = df12['Categoria 2'].unique().tolist()
categorias21 = df23['Categoria 2'].unique().tolist()
categorias22 = df23['Categoria 3'].unique().tolist()
categorias1 = categorias11 + categorias12
categorias2 = categorias21 + categorias22
categorias = categorias1 + categorias2
# Borrar duplicados
categorias = list(dict.fromkeys(categorias))
# Borrar nan
categorias = [x for x in categorias if str(x) != 'nan']
# Asignar un numero a cada categoria
categorias_dict = {}
for i in range(len(categorias)):
    categorias_dict[categorias[i]] = i

# Lista de colores rgb asociados a cada categoria
morados = [(226,206,246),(217,128,249),(175,80,229),(138,11,210),(91,6,114)]
colors = []
for i in range(len(categorias)):
    idx = i % len(morados)
    # Aplicar gama de morados y gris
    colors.append(f'rgba({morados[idx][0]}, {morados[idx][1]}, {morados[idx][2]}, {0.8})')

# Transformar las categorias a numeros
df12 = df12.replace(categorias_dict)
df23 = df23.replace(categorias_dict)

# Cambiar nombre de las columnas
df12stv = df12.rename(columns={'Categoria 1': 'source', 'Categoria 2': 'target', 'counts': 'value'})
df23stv = df23.rename(columns={'Categoria 2': 'source', 'Categoria 3': 'target', 'counts': 'value'})
# Concatenar los dos dataframes
df123 = pd.concat([df12stv, df23stv])
df123

C:\Users\sebag\AppData\Local\Temp\ipykernel_28920\2170094802.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12.loc[df12['Categoria 2'].isna(), 'Categoria 2'] = df12['Categoria 1']+ '2'
C:\Users\sebag\AppData\Local\Temp\ipykernel_28920\2170094802.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df23.loc[df23['Categoria 3'].isna(), 'Categoria 3'] = df23['Categoria 2']+ '3'


,source,target,value
0,0,6,1071
1,0,7,2699
2,0,8,8
3,0,9,71
4,0,10,2763
...,...,...,...
79,19,104,8
80,19,105,1
81,19,106,186
82,19,107,62


In [3]:
# Crear sankey diagram de muchos colores
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    valueformat = ".0f",
    valuesuffix = " units",
    arrangement='snap',
    node = dict(
        pad = 15,
        thickness = 20,
        line = dict(color = "grey", width = 0.8),
        label = categorias,
        color = colors
    ),
    link = dict(
        source = df123['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
        target = df123['target'],
        value = df123['value'],
        color = colors,
        arrowlen=15,
    ),
    textfont = dict(
        size = 8,
        color = 'white',
    )
    )])

fig.update_layout(
    title_text="Análisis Exploratorio de Datos <br>Source: Página principal de <a href='https://www.elpalaciodehierro.com/mundo-hogar.html'>El Palacio de Hierro</a>, sección Hogar",
    # font_size=10,
    width=1600,  # Ancho del diagrama
    height=900,   # Alto del diagrama
    font=dict(size = 15, color = 'white'),
    plot_bgcolor='black',
    paper_bgcolor='black',
    margin=dict(l=10, r=10, t=50, b=10),  # Ajustar los márgenes
)

# Guardar el gráfico como un archivo PNG
fig.write_image("sankey_diagram.png")

fig.show()

import plotly.express as px
fig.write_html("sankey_diagram.html")